In [ ]:
from qiskit import Aer, IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, execute
import socket, math

HOST = '127.0.0.1'
PORT = 64242

def quantumProcessing():
    device = Aer.get_backend('qasm_simulator')
    # set the number of samples used for statistics
    shots = 1024

    #Parameters
    nbPlayer = 2
    nbOfquanticToken = 2
        
    q = QuantumRegister(nbPlayer)
    c = ClassicalRegister(nbOfquanticToken)
    circuit = QuantumCircuit(q,c)
    circuit.h(q[0])
    circuit.cx(q[0],q[1])
    circuit.measure(q,c)
    
    job = execute(circuit, backend=device, shots=shots)
    data = job.result()

    # compile and run the quantum program
    job = execute(circuit, backend=device, shots=shots)
    data = ""
    res = job.result().get_counts(circuit)
    for key, value in res.items():
        data += key
        data += ";"
        data += str(value)
        data += ";"
    data += "!"
    return data

while(True):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        conn, addr = s.accept()
        with conn:
            print('Connected by', addr)
            while True:
                data = conn.recv(1024)
                data = quantumProcessing()
                print(data)
                conn.sendall(data.encode())